In [2]:
"""
os.chdir("C:\\Users\\podou\\Documents\\CU - DS\\Project")
os.getcwd()
"""

'\nos.chdir("C:\\Users\\podou\\Documents\\CU - DS\\Project")\nos.getcwd()\n'

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.model_selection import KFold, cross_val_score
from numpy import mean, absolute, sqrt
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")

#from mxnet import autograd, gluon, init, np, npx
#from mxnet.gluon import nn
#from d2l import mxnet as d2l
# The training dataset includes 1460 examples, 80 features, and 1 label, 
# while the test data contains 1459 examples and 80 features.

ModuleNotFoundError: No module named 'sklearn'

# Data cleaning

In [44]:
dataset_train = pd.read_csv('train.csv')
dataset_test = pd.read_csv('test.csv')

In [45]:
dataset_train

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL         65.0     8450   Pave   NaN      Reg   
1        2          20       RL         80.0     9600   Pave   NaN      Reg   
2        3          60       RL         68.0    11250   Pave   NaN      IR1   
3        4          70       RL         60.0     9550   Pave   NaN      IR1   
4        5          60       RL         84.0    14260   Pave   NaN      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1455  1456          60       RL         62.0     7917   Pave   NaN      Reg   
1456  1457          20       RL         85.0    13175   Pave   NaN      Reg   
1457  1458          70       RL         66.0     9042   Pave   NaN      Reg   
1458  1459          20       RL         68.0     9717   Pave   NaN      Reg   
1459  1460          20       RL         75.0     9937   Pave   NaN      Reg   

     LandContour Utilities  ... PoolArea PoolQC  Fence MiscFeature MiscVal  \
0            Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
1            Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
2            Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
3            Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
4            Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
...          ...       ...  ...      ...    ...    ...         ...     ...   
1455         Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
1456         Lvl    AllPub  ...        0    NaN  MnPrv         NaN       0   
1457         Lvl    AllPub  ...        0    NaN  GdPrv        Shed    2500   
1458         Lvl    AllPub  ...        0    NaN    NaN         NaN       0   
1459         Lvl    AllPub  ...        0    NaN    NaN         NaN       0   

     MoSold YrSold  SaleType  SaleCondition  SalePrice  
0         2   2008        WD         Normal     208500  
1         5   2007        WD         Normal     181500  
2         9   2008        WD         Normal     223500  
3         2   2006        WD        Abnorml     140000  
4        12   2008        WD         Normal     250000  
...     ...    ...       ...            ...        ...  
1455      8   2007        WD         Normal     175000  
1456      2   2010        WD         Normal     210000  
1457      5   2010        WD         Normal     266500  
1458      4   2010        WD         Normal     142125  
1459      6   2008        WD         Normal     147500  

[1460 rows x 81 columns]

In [46]:
dataset_test.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0  1461          20       RH         80.0    11622   Pave   NaN      Reg   
1  1462          20       RL         81.0    14267   Pave   NaN      IR1   
2  1463          60       RL         74.0    13830   Pave   NaN      IR1   
3  1464          60       RL         78.0     9978   Pave   NaN      IR1   
4  1465         120       RL         43.0     5005   Pave   NaN      IR1   

  LandContour Utilities  ... ScreenPorch PoolArea PoolQC  Fence MiscFeature  \
0         Lvl    AllPub  ...         120        0    NaN  MnPrv         NaN   
1         Lvl    AllPub  ...           0        0    NaN    NaN        Gar2   
2         Lvl    AllPub  ...           0        0    NaN  MnPrv         NaN   
3         Lvl    AllPub  ...           0        0    NaN    NaN         NaN   
4         HLS    AllPub  ...         144        0    NaN    NaN         NaN   

  MiscVal MoSold  YrSold  SaleType  SaleCondition  
0       0      6    2010        WD         Normal  
1   12500      6    2010        WD         Normal  
2       0      3    2010        WD         Normal  
3       0      6    2010        WD         Normal  
4       0      1    2010        WD         Normal  

[5 rows x 80 columns]

# Preprocess data

In [47]:
price = dataset_train["SalePrice"]
all = pd.concat([dataset_train, dataset_test]).iloc[:,0:79]
print(all.dtypes[0:5])
#second column MSSubClass is categorical feature but is considered as 
#int so we remapped it

all["MSSubClass"] = all["MSSubClass"].astype(str)
all["MSSubClass"] = all["MSSubClass"].replace(["20","30","40","45","50","60","70","75","80","85","90","120","150"
                           , "160","180","190"], ["1-story_1946+","1-story_1945-", "1-story_with_finished_attic"
                           ,"1.5_story_unifinished", "1.5_finished", "2-story_1946+", "2-story_1945-",
                            "2.5_story", "split_or_multi_level", "split_foyer", "1-story_PUD",  "duplex",
                            "1.5_story_PUD", "2-story_PUD", "PUD_Multilevel","2_family_conversion"])
all["MSSubClass"].head()

Id               int64
MSSubClass       int64
MSZoning        object
LotFrontage    float64
LotArea          int64
dtype: object


0    2-story_1946+
1    1-story_1946+
2    2-story_1946+
3    2-story_1945-
4    2-story_1946+
Name: MSSubClass, dtype: object

In [48]:
#modify yrs
#YearBuilt: Original construction date
#YearRemodAdd: Remodel date (same as construction date if no remodeling or additions)
#YrSold: Year Sold (YYYY)

# remodel 0 = no 1 =yes
# age: yr-sold - max(yearbulit, yearromodadd)

remod = all["YearBuilt"] == all["YearRemodAdd"]


remod = np.multiply(np.array(remod),1)
        
#HouseAge = all["YearBuilt"] - max(all["YearBuilt"], all["YearRemodAdd"])

HouseAge = all["YrSold"] - all[["YearBuilt", "YearRemodAdd"]].max(axis=1)
print(min(HouseAge))

all["HouseAge"] = HouseAge
all["remod"] = remod
all = all.drop(columns=["YearBuilt","YearRemodAdd","YrSold" ])
all


-2


Id     MSSubClass MSZoning  LotFrontage  LotArea Street Alley  \
0        1  2-story_1946+       RL         65.0     8450   Pave   NaN   
1        2  1-story_1946+       RL         80.0     9600   Pave   NaN   
2        3  2-story_1946+       RL         68.0    11250   Pave   NaN   
3        4  2-story_1945-       RL         60.0     9550   Pave   NaN   
4        5  2-story_1946+       RL         84.0    14260   Pave   NaN   
...    ...            ...      ...          ...      ...    ...   ...   
1454  2915    2-story_PUD       RM         21.0     1936   Pave   NaN   
1455  2916    2-story_PUD       RM         21.0     1894   Pave   NaN   
1456  2917  1-story_1946+       RL        160.0    20000   Pave   NaN   
1457  2918    split_foyer       RL         62.0    10441   Pave   NaN   
1458  2919  2-story_1946+       RL         74.0     9627   Pave   NaN   

     LotShape LandContour Utilities  ... ScreenPorch PoolArea PoolQC  Fence  \
0         Reg         Lvl    AllPub  ...           0        0    NaN    NaN   
1         Reg         Lvl    AllPub  ...           0        0    NaN    NaN   
2         IR1         Lvl    AllPub  ...           0        0    NaN    NaN   
3         IR1         Lvl    AllPub  ...           0        0    NaN    NaN   
4         IR1         Lvl    AllPub  ...           0        0    NaN    NaN   
...       ...         ...       ...  ...         ...      ...    ...    ...   
1454      Reg         Lvl    AllPub  ...           0        0    NaN    NaN   
1455      Reg         Lvl    AllPub  ...           0        0    NaN    NaN   
1456      Reg         Lvl    AllPub  ...           0        0    NaN    NaN   
1457      Reg         Lvl    AllPub  ...           0        0    NaN  MnPrv   
1458      Reg         Lvl    AllPub  ...           0        0    NaN    NaN   

     MiscFeature MiscVal MoSold  SaleType  HouseAge remod  
0            NaN       0      2        WD         5     1  
1            NaN       0      5        WD        31     1  
2            NaN       0      9        WD         6     0  
3            NaN       0      2        WD        36     0  
4            NaN       0     12        WD         8     1  
...          ...     ...    ...       ...       ...   ...  
1454         NaN       0      6        WD        36     1  
1455         NaN       0      4        WD        36     1  
1456         NaN       0      9        WD        10     0  
1457        Shed     700      7        WD        14     1  
1458         NaN       0     11        WD        12     0  

[2919 rows x 78 columns]

In [49]:
#BsmtFullBath: Basement full bathrooms
#BsmtHalfBath: Basement half bathrooms
#FullBath: Full bathrooms above grade
#HalfBath: Half baths above grade

#A full bathroom is made up of four parts: a sink, a shower, a bathtub, and a toilet. Anything less than that, and you can't officially consider it a full bath
#A bathroom with just a sink and a toilet is a half-bath.

all['BsmtFullBath']  = all['BsmtFullBath'].fillna(0)
all['BsmtHalfBath']  = all['BsmtHalfBath'].fillna(0)

all['NumBath'] = all['BsmtFullBath'] + all['BsmtHalfBath'] *.5 + all['HalfBath'] + all['HalfBath'] * .5

all = all.drop(columns=['BsmtFullBath','BsmtHalfBath','HalfBath','HalfBath' ])

all

Id     MSSubClass MSZoning  LotFrontage  LotArea Street Alley  \
0        1  2-story_1946+       RL         65.0     8450   Pave   NaN   
1        2  1-story_1946+       RL         80.0     9600   Pave   NaN   
2        3  2-story_1946+       RL         68.0    11250   Pave   NaN   
3        4  2-story_1945-       RL         60.0     9550   Pave   NaN   
4        5  2-story_1946+       RL         84.0    14260   Pave   NaN   
...    ...            ...      ...          ...      ...    ...   ...   
1454  2915    2-story_PUD       RM         21.0     1936   Pave   NaN   
1455  2916    2-story_PUD       RM         21.0     1894   Pave   NaN   
1456  2917  1-story_1946+       RL        160.0    20000   Pave   NaN   
1457  2918    split_foyer       RL         62.0    10441   Pave   NaN   
1458  2919  2-story_1946+       RL         74.0     9627   Pave   NaN   

     LotShape LandContour Utilities  ... PoolArea PoolQC  Fence MiscFeature  \
0         Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
1         Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
2         IR1         Lvl    AllPub  ...        0    NaN    NaN         NaN   
3         IR1         Lvl    AllPub  ...        0    NaN    NaN         NaN   
4         IR1         Lvl    AllPub  ...        0    NaN    NaN         NaN   
...       ...         ...       ...  ...      ...    ...    ...         ...   
1454      Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
1455      Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
1456      Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
1457      Reg         Lvl    AllPub  ...        0    NaN  MnPrv        Shed   
1458      Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   

     MiscVal MoSold SaleType  HouseAge  remod NumBath  
0          0      2       WD         5      1     2.5  
1          0      5       WD        31      1     0.5  
2          0      9       WD         6      0     2.5  
3          0      2       WD        36      0     1.0  
4          0     12       WD         8      1     2.5  
...      ...    ...      ...       ...    ...     ...  
1454       0      6       WD        36      1     1.5  
1455       0      4       WD        36      1     1.5  
1456       0      9       WD        10      0     1.0  
1457     700      7       WD        14      1     0.5  
1458       0     11       WD        12      0     1.5  

[2919 rows x 76 columns]

In [50]:
#garage variables

price

all['GarageArea']  = all['GarageArea'].fillna(0)
all['GarageCars']  = all['GarageCars'].fillna(0)

cor_Garage_area_cars = all['GarageArea'].corr(all['GarageCars'])
print("Since GarageArea and GarageCars are highly correlated (cor = ", round(cor_Garage_area_cars,4), ")\n")
print("in case of parsimony, we choose one of them has higher correlation with SalePrice")

print("correlation between GarageArea and SalePrice is", round(all['GarageArea'][0:1459].corr(price),4))
print("correlation between GarageCars and SalePrice is", round(all['GarageCars'][0:1459].corr(price),4))
print("so we keep the GarageCars variable")

all = all.drop(columns=['GarageArea' ])

all

Since GarageArea and GarageCars are highly correlated (cor =  0.8899 )

in case of parsimony, we choose one of them has higher correlation with SalePrice
correlation between GarageArea and SalePrice is 0.6234
correlation between GarageCars and SalePrice is 0.6404
so we keep the GarageCars variable


Id     MSSubClass MSZoning  LotFrontage  LotArea Street Alley  \
0        1  2-story_1946+       RL         65.0     8450   Pave   NaN   
1        2  1-story_1946+       RL         80.0     9600   Pave   NaN   
2        3  2-story_1946+       RL         68.0    11250   Pave   NaN   
3        4  2-story_1945-       RL         60.0     9550   Pave   NaN   
4        5  2-story_1946+       RL         84.0    14260   Pave   NaN   
...    ...            ...      ...          ...      ...    ...   ...   
1454  2915    2-story_PUD       RM         21.0     1936   Pave   NaN   
1455  2916    2-story_PUD       RM         21.0     1894   Pave   NaN   
1456  2917  1-story_1946+       RL        160.0    20000   Pave   NaN   
1457  2918    split_foyer       RL         62.0    10441   Pave   NaN   
1458  2919  2-story_1946+       RL         74.0     9627   Pave   NaN   

     LotShape LandContour Utilities  ... PoolArea PoolQC  Fence MiscFeature  \
0         Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
1         Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
2         IR1         Lvl    AllPub  ...        0    NaN    NaN         NaN   
3         IR1         Lvl    AllPub  ...        0    NaN    NaN         NaN   
4         IR1         Lvl    AllPub  ...        0    NaN    NaN         NaN   
...       ...         ...       ...  ...      ...    ...    ...         ...   
1454      Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
1455      Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
1456      Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
1457      Reg         Lvl    AllPub  ...        0    NaN  MnPrv        Shed   
1458      Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   

     MiscVal MoSold SaleType  HouseAge  remod NumBath  
0          0      2       WD         5      1     2.5  
1          0      5       WD        31      1     0.5  
2          0      9       WD         6      0     2.5  
3          0      2       WD        36      0     1.0  
4          0     12       WD         8      1     2.5  
...      ...    ...      ...       ...    ...     ...  
1454       0      6       WD        36      1     1.5  
1455       0      4       WD        36      1     1.5  
1456       0      9       WD        10      0     1.0  
1457     700      7       WD        14      1     0.5  
1458       0     11       WD        12      0     1.5  

[2919 rows x 75 columns]

In [51]:
#x1stFlrSF and totalbasement
#x1stF   + 2nd = grlivarea

df_test1 = all[['1stFlrSF', '2ndFlrSF', 'GrLivArea']]
df_test1['sum1and2'] = df_test1['1stFlrSF'] + df_test1['2ndFlrSF']
df_test1['DiffSumAndTot'] = df_test1['GrLivArea'] - df_test1['sum1and2'] 
print(df_test1[1:20])

print("we can see there are only", sum((all['1stFlrSF'] + all['2ndFlrSF']) != all['GrLivArea']) ,"out of 2919 data sets, approximately 1% of total data set\n",
      "that first floor area + second floor area != totol above ground living area, probably there are some houses has\n",
      "> 2 floors that has not been taken into account of the dataset")
print("so we comes to the conclusion that Total above ground living area('GrLivArea') explains both two variables\n")
print("first floor sf and second floor sq")
print("so we drop those two features")

all = all.drop(columns=['1stFlrSF','2ndFlrSF' ])

all

    1stFlrSF  2ndFlrSF  GrLivArea  sum1and2  DiffSumAndTot
1       1262         0       1262      1262              0
2        920       866       1786      1786              0
3        961       756       1717      1717              0
4       1145      1053       2198      2198              0
5        796       566       1362      1362              0
6       1694         0       1694      1694              0
7       1107       983       2090      2090              0
8       1022       752       1774      1774              0
9       1077         0       1077      1077              0
10      1040         0       1040      1040              0
11      1182      1142       2324      2324              0
12       912         0        912       912              0
13      1494         0       1494      1494              0
14      1253         0       1253      1253              0
15       854         0        854       854              0
16      1004         0       1004      1004             

Id     MSSubClass MSZoning  LotFrontage  LotArea Street Alley  \
0        1  2-story_1946+       RL         65.0     8450   Pave   NaN   
1        2  1-story_1946+       RL         80.0     9600   Pave   NaN   
2        3  2-story_1946+       RL         68.0    11250   Pave   NaN   
3        4  2-story_1945-       RL         60.0     9550   Pave   NaN   
4        5  2-story_1946+       RL         84.0    14260   Pave   NaN   
...    ...            ...      ...          ...      ...    ...   ...   
1454  2915    2-story_PUD       RM         21.0     1936   Pave   NaN   
1455  2916    2-story_PUD       RM         21.0     1894   Pave   NaN   
1456  2917  1-story_1946+       RL        160.0    20000   Pave   NaN   
1457  2918    split_foyer       RL         62.0    10441   Pave   NaN   
1458  2919  2-story_1946+       RL         74.0     9627   Pave   NaN   

     LotShape LandContour Utilities  ... PoolArea PoolQC  Fence MiscFeature  \
0         Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
1         Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
2         IR1         Lvl    AllPub  ...        0    NaN    NaN         NaN   
3         IR1         Lvl    AllPub  ...        0    NaN    NaN         NaN   
4         IR1         Lvl    AllPub  ...        0    NaN    NaN         NaN   
...       ...         ...       ...  ...      ...    ...    ...         ...   
1454      Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
1455      Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
1456      Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
1457      Reg         Lvl    AllPub  ...        0    NaN  MnPrv        Shed   
1458      Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   

     MiscVal MoSold SaleType  HouseAge  remod NumBath  
0          0      2       WD         5      1     2.5  
1          0      5       WD        31      1     0.5  
2          0      9       WD         6      0     2.5  
3          0      2       WD        36      0     1.0  
4          0     12       WD         8      1     2.5  
...      ...    ...      ...       ...    ...     ...  
1454       0      6       WD        36      1     1.5  
1455       0      4       WD        36      1     1.5  
1456       0      9       WD        10      0     1.0  
1457     700      7       WD        14      1     0.5  
1458       0     11       WD        12      0     1.5  

[2919 rows x 73 columns]

In [52]:
print("since Total # of rooms and above ground living area are highly correlated, cor = ", round(all['TotRmsAbvGrd'].corr(all['GrLivArea']),4))
print("so we choose one has higher correlation with SalePrice")
all = all.drop(columns=['TotRmsAbvGrd' ])
all



since Total # of rooms and above ground living area are highly correlated, cor =  0.8084
so we choose one has higher correlation with SalePrice


Id     MSSubClass MSZoning  LotFrontage  LotArea Street Alley  \
0        1  2-story_1946+       RL         65.0     8450   Pave   NaN   
1        2  1-story_1946+       RL         80.0     9600   Pave   NaN   
2        3  2-story_1946+       RL         68.0    11250   Pave   NaN   
3        4  2-story_1945-       RL         60.0     9550   Pave   NaN   
4        5  2-story_1946+       RL         84.0    14260   Pave   NaN   
...    ...            ...      ...          ...      ...    ...   ...   
1454  2915    2-story_PUD       RM         21.0     1936   Pave   NaN   
1455  2916    2-story_PUD       RM         21.0     1894   Pave   NaN   
1456  2917  1-story_1946+       RL        160.0    20000   Pave   NaN   
1457  2918    split_foyer       RL         62.0    10441   Pave   NaN   
1458  2919  2-story_1946+       RL         74.0     9627   Pave   NaN   

     LotShape LandContour Utilities  ... PoolArea PoolQC  Fence MiscFeature  \
0         Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
1         Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
2         IR1         Lvl    AllPub  ...        0    NaN    NaN         NaN   
3         IR1         Lvl    AllPub  ...        0    NaN    NaN         NaN   
4         IR1         Lvl    AllPub  ...        0    NaN    NaN         NaN   
...       ...         ...       ...  ...      ...    ...    ...         ...   
1454      Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
1455      Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
1456      Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   
1457      Reg         Lvl    AllPub  ...        0    NaN  MnPrv        Shed   
1458      Reg         Lvl    AllPub  ...        0    NaN    NaN         NaN   

     MiscVal MoSold SaleType  HouseAge  remod NumBath  
0          0      2       WD         5      1     2.5  
1          0      5       WD        31      1     0.5  
2          0      9       WD         6      0     2.5  
3          0      2       WD        36      0     1.0  
4          0     12       WD         8      1     2.5  
...      ...    ...      ...       ...    ...     ...  
1454       0      6       WD        36      1     1.5  
1455       0      4       WD        36      1     1.5  
1456       0      9       WD        10      0     1.0  
1457     700      7       WD        14      1     0.5  
1458       0     11       WD        12      0     1.5  

[2919 rows x 72 columns]

# Preprocess data END


In [53]:
all_features = all

# replacing all missing values by the corresponding feature’s mean. 
# Then, to put all features on a common scale, 
# we standardize the data by rescaling features to zero mean and unit variance
# Intuitively, we standardize the data for two reasons. 
# First, it proves convenient for optimization. 
# Second, because we do not know a priori which features will be relevant,
# we do not want to penalize coefficients assigned to one feature 
# more than on any other.


# If test data were inaccessible, mean and standard deviation could be
# calculated from training data
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# After standardizing the data all means vanish, hence we can set missing
# values to 0
all_features[numeric_features] = all_features[numeric_features].fillna(0)

In [54]:
# Next we deal with discrete values. 
# We replace them by a one-hot encoding
# Dummy variables
# `Dummy_na=True` considers "na" (missing value) as a valid feature value, and
# creates an indicator feature for it
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape

(2919, 334)

In [55]:
all_features = all_features.iloc[:,1:] # exclude id

In [57]:
train = all_features.iloc[0:1460,]
train


LotFrontage   LotArea  OverallQual  OverallCond  MasVnrArea  BsmtFinSF1  \
0       -0.184443 -0.217841     0.646073    -0.507197    0.523038    0.580708   
1        0.458096 -0.072032    -0.063174     2.187904   -0.569893    1.177709   
2       -0.055935  0.137173     0.646073    -0.507197    0.333448    0.097840   
3       -0.398622 -0.078371     0.646073    -0.507197   -0.569893   -0.494771   
4        0.629439  0.518814     1.355319    -0.507197    1.381770    0.468770   
...           ...       ...          ...          ...         ...         ...   
1455    -0.312950 -0.285421    -0.063174    -0.507197   -0.569893   -0.968860   
1456     0.672275  0.381246    -0.063174     0.391170    0.093673    0.765076   
1457    -0.141607 -0.142781     0.646073     3.086271   -0.569893   -0.365275   
1458    -0.055935 -0.057197    -0.772420     0.391170   -0.569893   -0.861312   
1459     0.243916 -0.029303    -0.772420     0.391170   -0.569893    0.852870   

      BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  LowQualFinSF  ...  SaleType_COD  \
0      -0.293030  -0.934542    -0.444176      -0.10118  ...             0   
1      -0.293030  -0.629681     0.476948      -0.10118  ...             0   
2      -0.293030  -0.288418    -0.298974      -0.10118  ...             0   
3      -0.293030  -0.047258    -0.671053      -0.10118  ...             0   
4      -0.293030  -0.161013     0.211501      -0.10118  ...             0   
...          ...        ...          ...           ...  ...           ...   
1455   -0.293030   0.892353    -0.224104      -0.10118  ...             0   
1456    0.670295   0.064221     1.112205      -0.10118  ...             0   
1457   -0.293030   0.719446     0.227382      -0.10118  ...             0   
1458    5.788329  -1.275805     0.059493      -0.10118  ...             0   
1459    1.420862  -0.966393     0.463335      -0.10118  ...             0   

      SaleType_CWD  SaleType_Con  SaleType_ConLD  SaleType_ConLI  \
0                0             0               0               0   
1                0             0               0               0   
2                0             0               0               0   
3                0             0               0               0   
4                0             0               0               0   
...            ...           ...             ...             ...   
1455             0             0               0               0   
1456             0             0               0               0   
1457             0             0               0               0   
1458             0             0               0               0   
1459             0             0               0               0   

      SaleType_ConLw  SaleType_New  SaleType_Oth  SaleType_WD  SaleType_nan  
0                  0             0             0            1             0  
1                  0             0             0            1             0  
2                  0             0             0            1             0  
3                  0             0             0            1             0  
4                  0             0             0            1             0  
...              ...           ...           ...          ...           ...  
1455               0             0             0            1             0  
1456               0             0             0            1             0  
1457               0             0             0            1             0  
1458               0             0             0            1             0  
1459               0             0             0            1             0  

[1460 rows x 333 columns]

In [58]:
test = all_features.iloc[1460:,]
test

LotFrontage   LotArea  OverallQual  OverallCond  MasVnrArea  BsmtFinSF1  \
0        0.458096  0.184340    -0.772420     0.391170   -0.569893    0.058332   
1        0.500932  0.519702    -0.063174     0.391170    0.032335    1.056991   
2        0.201080  0.464294    -0.772420    -0.507197   -0.569893    0.767271   
3        0.372424 -0.024105    -0.063174     0.391170   -0.458369    0.352443   
4       -1.126832 -0.654636     1.355319    -0.507197   -0.569893   -0.391613   
...           ...       ...          ...          ...         ...         ...   
1454    -2.069222 -1.043758    -1.481667     1.289537   -0.569893   -0.968860   
1455    -2.069222 -1.049083    -1.481667    -0.507197   -0.569893   -0.415757   
1456     3.884968  1.246594    -0.772420     1.289537   -0.569893    1.717643   
1457    -0.312950  0.034599    -0.772420    -0.507197   -0.569893   -0.229194   
1458     0.201080 -0.068608     0.646073    -0.507197   -0.045732    0.694840   

      BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  LowQualFinSF  ...  SaleType_COD  \
0       0.558006  -0.661532    -0.385187      -0.10118  ...             0   
1      -0.293030  -0.352120     0.628956      -0.10118  ...             0   
2      -0.293030  -0.964118    -0.280824      -0.10118  ...             0   
3      -0.293030  -0.538677    -0.285361      -0.10118  ...             0   
4      -0.293030   1.037958     0.517786      -0.10118  ...             0   
...          ...        ...          ...           ...  ...           ...   
1454   -0.293030  -0.033608    -1.147496      -0.10118  ...             0   
1455   -0.293030  -0.606930    -1.147496      -0.10118  ...             0   
1456   -0.293030  -1.275805     0.390734      -0.10118  ...             0   
1457   -0.293030   0.032370    -0.317124      -0.10118  ...             0   
1458   -0.293030  -0.734335    -0.126547      -0.10118  ...             0   

      SaleType_CWD  SaleType_Con  SaleType_ConLD  SaleType_ConLI  \
0                0             0               0               0   
1                0             0               0               0   
2                0             0               0               0   
3                0             0               0               0   
4                0             0               0               0   
...            ...           ...             ...             ...   
1454             0             0               0               0   
1455             0             0               0               0   
1456             0             0               0               0   
1457             0             0               0               0   
1458             0             0               0               0   

      SaleType_ConLw  SaleType_New  SaleType_Oth  SaleType_WD  SaleType_nan  
0                  0             0             0            1             0  
1                  0             0             0            1             0  
2                  0             0             0            1             0  
3                  0             0             0            1             0  
4                  0             0             0            1             0  
...              ...           ...           ...          ...           ...  
1454               0             0             0            1             0  
1455               0             0             0            1             0  
1456               0             0             0            1             0  
1457               0             0             0            1             0  
1458               0             0             0            1             0  

[1459 rows x 333 columns]

# Data Analyzing and Model Selection 